In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine-Review-Analysis").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Baby_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    9970739| R8EWA1OFT84NX|B00GSP5D94|     329991347|Summer Infant Swa...|            Baby|          5|            0|          0|   N|                Y|Great swaddled bl...|Loved these swadd...| 2015-08-31|
|         US|   23538442|R2JWY4YRQD4FOP|B00YYDDZGU|     646108902|Pacifier Clip Gir...|            Baby|          5|    

In [4]:
# Create the vine DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R8EWA1OFT84NX|          5|            0|          0|   N|                Y|
|R2JWY4YRQD4FOP|          5|            0|          0|   N|                N|
| RL5ESX231LZ0B|          5|            0|          0|   N|                Y|
| RRMS9ZWJ2KD08|          5|            0|          0|   N|                Y|
|R14I3ZG5E6S7YM|          5|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [8]:
# create a dataframe where total votes are greater than or equal 20
total_votes_df = vine_df.filter(vine_df.total_votes >= 20)
total_votes_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| ROI00QN8IS49I|          5|           83|         91|   N|                N|
|R13C5INE1RTZP6|          2|           26|         26|   N|                Y|
| RXVMMXCL67MZN|          4|          378|        383|   N|                Y|
|R33JJQWAUYBKD3|          5|          270|        280|   N|                Y|
|R3N0XV9267NOXV|          5|           41|         47|   Y|                N|
|R31HQD6YXSQV1W|          3|           40|         48|   N|                Y|
|R33LQSF958O6K8|          5|           39|         41|   N|                Y|
|R3OIDSQJ84W7J1|          5|           21|         24|   N|                N|
|R3UUR313K5VVTL|          1|            1|         24|   N|                Y|
| RODE8K12S7148|          5|           41|         45|   N|     

In [9]:
# create a datafram to retrieve all the rows where number of helpful votes divided by total votes is greater than or equal to 50%
helpful_votes_df = total_votes_df.filter((total_votes_df.helpful_votes / total_votes_df.total_votes) >= 0.5)
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| ROI00QN8IS49I|          5|           83|         91|   N|                N|
|R13C5INE1RTZP6|          2|           26|         26|   N|                Y|
| RXVMMXCL67MZN|          4|          378|        383|   N|                Y|
|R33JJQWAUYBKD3|          5|          270|        280|   N|                Y|
|R3N0XV9267NOXV|          5|           41|         47|   Y|                N|
|R31HQD6YXSQV1W|          3|           40|         48|   N|                Y|
|R33LQSF958O6K8|          5|           39|         41|   N|                Y|
|R3OIDSQJ84W7J1|          5|           21|         24|   N|                N|
| RODE8K12S7148|          5|           41|         45|   N|                Y|
|R2QZFYFUKP22SP|          5|           33|         37|   N|     

In [13]:
# create a dataframe with paid reviews
paid_df = helpful_votes_df.filter(helpful_votes_df.vine == 'Y')
paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3N0XV9267NOXV|          5|           41|         47|   Y|                N|
| RSA6JQ346JZHZ|          5|           55|         64|   Y|                Y|
|R1FXF4HRMCLG4C|          5|           69|         84|   Y|                N|
| RCTBWC3II42MG|          4|          113|        117|   Y|                N|
| RTMQM2CQ1XIZ0|          5|           21|         25|   Y|                N|
|R2CBJLCKQ612KU|          4|           27|         35|   Y|                N|
|R322QDGO4AV2B5|          5|           55|         65|   Y|                N|
| R2S7CXRDGQ6EE|          4|           36|         39|   Y|                N|
|R3ECMXK0SGR1VV|          5|           59|         64|   Y|                N|
|R3D55CPJ6J6Z78|          4|           15|         20|   Y|     

In [12]:
# created a dataframe with unpaid reviews
unpaid_df = helpful_votes_df.filter(helpful_votes_df.vine == 'N')
unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| ROI00QN8IS49I|          5|           83|         91|   N|                N|
|R13C5INE1RTZP6|          2|           26|         26|   N|                Y|
| RXVMMXCL67MZN|          4|          378|        383|   N|                Y|
|R33JJQWAUYBKD3|          5|          270|        280|   N|                Y|
|R31HQD6YXSQV1W|          3|           40|         48|   N|                Y|
|R33LQSF958O6K8|          5|           39|         41|   N|                Y|
|R3OIDSQJ84W7J1|          5|           21|         24|   N|                N|
| RODE8K12S7148|          5|           41|         45|   N|                Y|
|R2QZFYFUKP22SP|          5|           33|         37|   N|                Y|
| RFON10GAZKDL3|          3|           16|         20|   N|     

In [19]:
# Calculate total number of reviews, number of 5 star reviews and % of 5 star reviews for paid dataframe.
total_paid_reviews = paid_df.count()
paid_five_star_reviews = paid_df.filter(paid_df.star_rating == 5).count()
paid_five_star_percent = (paid_five_star_reviews / total_paid_reviews) * 100

print(f"The total number of paid reviews is {total_paid_reviews}.")
print(f"The total number of paid 5 star reviews is {paid_five_star_reviews}.")
print(f"The percentage of paid 5 star reviews is {paid_five_star_percent}%.")

The total number of paid reviews is 463.
The total number of paid 5 star reviews is 202.
The percentage of paid 5 star reviews is 43.628509719222464%.


In [20]:
# Calculate the total number of reviews, number of 5 star reviews, and % of 5 star reviews for the unpaid dataframe.
total_unpaid_reviews = unpaid_df.count()
unpaid_five_star_reviews = unpaid_df.filter(unpaid_df.star_rating == 5).count()
unpaid_five_star_percent = (unpaid_five_star_reviews / total_unpaid_reviews) * 100

print(f"The total number of unpaid reviews is {total_unpaid_reviews}.")
print(f"The total number of unpaid 5 star reviews is {unpaid_five_star_reviews}.")
print(f"The percentage of unpaid 5 star reviews is {unpaid_five_star_percent}%.")

The total number of unpaid reviews is 25094.
The total number of unpaid 5 star reviews is 12033.
The percentage of unpaid 5 star reviews is 47.95170160197657%.
